In [1]:
using Pkg
Pkg.activate(".")

using Dates
using Random
using Printf

using Oceananigans
using Oceananigans.Units: minute, minutes, hour, hours, day, days, meter, meters, kilometer, kilometers


  Activating new project at `~/Desktop/Minor/NEW/PhysicsInformedNN_Alkalinity/notebooks`


In [2]:
# --------------------------------------------------
# computing architecture
# --------------------------------------------------
architecture = CPU()

CPU()

In [3]:
# --------------------------------------------------
# grid
# --------------------------------------------------
const Nx = 100  # number of x points 
const Ny = 100  # number of y points 
const Nz = 1    # number of z points 

const Lx = 100kilometers # (m) domain x extent
const Ly = 100kilometers # (m) domain y extent
const Lz = 10meters      # (m) domain depth

grid = RectilinearGrid(
    architecture, 
    topology = (Periodic, Periodic, Bounded),
    size = (Nx, Nx, Nz),
    halo = (3,3,2),
    x = (0, Lx),
    y = (0, Ly),
    z = (-Lz, 0)
)

100×100×1 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×2 halo
├── Periodic x ∈ [0.0, 100000.0) regularly spaced with Δx=1000.0
├── Periodic y ∈ [0.0, 100000.0) regularly spaced with Δy=1000.0
└── Bounded  z ∈ [-10.0, 0.0]    regularly spaced with Δz=10.0

In [4]:
# --------------------------------------------------
# coriolis
# --------------------------------------------------
coriolis = FPlane(f=1e-4)

# --------------------------------------------------
# closure
# --------------------------------------------------
closure = AnisotropicMinimumDissipation()

# --------------------------------------------------
# seawater buoyancy
# --------------------------------------------------
buoyancy = SeawaterBuoyancy(
    equation_of_state = LinearEquationOfState(
        thermal_expansion = 2e-4,
        haline_contraction = 8e-4)
)


SeawaterBuoyancy{Float64}:
├── gravitational_acceleration: 9.80665
└── equation_of_state: LinearEquationOfState(thermal_expansion=0.0002, haline_contraction=0.0008)

In [5]:
# --------------------------------------------------
# surface stress (gaussian)
# --------------------------------------------------
u₁₀ = 10    # m s⁻¹, average wind velocity 10 meters above the ocean
ρₒ = 1026.0 # kg m⁻³, average density at the surface of the world ocean
cᴰ = 2.5e-3 # dimensionless drag coefficient
ρₐ = 1.225  # kg m⁻³, average density of air at sea-level

x₀ = Lx / 2          # center of Gaussian in x
y₀ = Ly / 2          # center of Gaussian in y
σ  = 10kilometers     # standard deviation of the Gaussian

# Wind is blowing in x-direction, varying only along y
@inline one_direction_gaussian_wind_stress(x, y, t, p) = exp(-((y - y₀)^2) / (2 * σ^2))

@inline wind_stress_x(x, y, t, p) = (- ρₐ / ρₒ * cᴰ * u₁₀^2) * one_direction_gaussian_wind_stress(x, y, t, p)
@inline wind_stress_y(x, y, t, p) = 0.0


wind_stress_y (generic function with 1 method)

In [6]:
# --------------------------------------------------
# boundary conditions
# --------------------------------------------------
u_top_bcs = FluxBoundaryCondition(wind_stress_x, parameters=(k=4π, ω=3.0, τ=1e-4))
v_top_bcs = FluxBoundaryCondition(wind_stress_y, parameters=(k=4π, ω=3.0, τ=1e-4))

u_bcs = FieldBoundaryConditions(top = u_top_bcs) 
v_bcs = FieldBoundaryConditions(top = v_top_bcs)

T_bcs = FieldBoundaryConditions()
S_bcs = FieldBoundaryConditions()
c_bcs = FieldBoundaryConditions()

Oceananigans.FieldBoundaryConditions, with boundary conditions
├── west: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── east: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── south: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── north: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── bottom: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
├── top: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)
└── immersed: DefaultBoundaryCondition (FluxBoundaryCondition: Nothing)

In [7]:
# --------------------------------------------------
# instantiate model
# --------------------------------------------------
model = NonhydrostaticModel(; grid, buoyancy,
                            timestepper = :QuasiAdamsBashforth2,
                            advection = WENO(),
                            tracers = (:T, :S, :c),
                            coriolis = coriolis,
                            closure = closure,
                            boundary_conditions = (u=u_bcs, v=v_bcs, T=T_bcs, S=S_bcs, c=c_bcs))



[ Info: Using the advection scheme UpwindBiased(order=1) in the z-direction because size(grid, 3) = 1


NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 100×100×1 RectilinearGrid{Float64, Periodic, Periodic, Bounded} on CPU with 3×3×2 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── advection scheme: FluxFormAdvection(x=WENO(order=5), y=WENO(order=5), z=UpwindBiased(order=1))
├── tracers: (T, S, c)
├── closure: AnisotropicMinimumDissipation{Oceananigans.TurbulenceClosures.ExplicitTimeDiscretization, @NamedTuple{T::Float64, S::Float64, c::Float64}, Float64, Nothing}
├── buoyancy: SeawaterBuoyancy with g=9.80665 and LinearEquationOfState(thermal_expansion=0.0002, haline_contraction=0.0008) with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [8]:
# --------------------------------------------------
# initial conditions
# --------------------------------------------------
const u_initial = 0.0
const v_initial = 0.0
const T_initial = 20.0
const S_initial = 35.0

# initial concentration value
# center indices (x, y) 
center_x_index = Int(grid.Nx / 2)  # center index in the x-direction
center_y_index = Int(grid.Ny / 2)  # center index in the y-direction
surface_z_index = grid.Nz          # index of surface z grid cell 

# initialize tracer values to zero
c_initial = model.tracers.c 

# grid cell volume
# note: this assumes grid cells all same size
dx = Lx / Nx       # (m) grid cell x length 
dy = Ly / Ny       # (m) grid cell y length
dz = Lz / Nz       # (m) grid cell z length
dv = dx * dy * dz  # (m3) grid cell volume

# set initial concentration to 1 mol / m3 at center grid cell
c_initial[center_x_index, center_y_index, surface_z_index] = 1

set!(model, u=u_initial, v=v_initial, T=T_initial, S=S_initial, c=c_initial)

In [9]:
# --------------------------------------------------
# simulation and outputs
# --------------------------------------------------
simulation = Simulation(model, Δt=10minutes, stop_time=100days)

wizard = TimeStepWizard(cfl=0.2)

simulation.callbacks[:wizard] = Callback(wizard, IterationInterval(10))


progress_message(sim) = @printf("Iteration: %04d, time: %s, Δt: %s, max(|u|) = %.1e ms⁻¹, wall time: %s\n",
                                iteration(sim), prettytime(sim), prettytime(sim.Δt),
                                maximum(abs, sim.model.velocities.u), prettytime(sim.run_wall_time))

add_callback!(simulation, progress_message, IterationInterval(20))



In [10]:
# --------------------------------------------------
# output writer
# --------------------------------------------------
filename = "output-tracer-release-gaussian-wind_$(string(today()))"

simulation.output_writers[:surface_slice_writer] = NetCDFOutputWriter(
    model, 
    (; model.velocities.u, model.velocities.v, model.tracers.S, model.tracers.T, model.tracers.c),
    filename = filename * ".nc",
    schedule = AveragedTimeInterval(1hour, window=1hour),
    indices=(:, :, grid.Nz),  
    overwrite_existing = true 
)

NetCDFOutputWriter scheduled on TimeInterval(1 hour):
├── filepath: output-tracer-release-gaussian-wind_2025-06-18.nc
├── dimensions: zC(1), zF(1), xC(100), yF(100), xF(100), yC(100), time(0)
├── 5 outputs: (v, S, c, T, u) averaged on AveragedTimeInterval(window=1 hour, stride=1, interval=1 hour)
└── array type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 19.4 KiB

In [11]:
# --------------------------------------------------
# run simulation
# --------------------------------------------------
run!(simulation)

[ Info: Initializing simulation...


Iteration: 0000, time: 0 seconds, Δt: 11 minutes, max(|u|) = 0.0e+00 ms⁻¹, wall time: 0 seconds


[ Info:     ... simulation initialization complete (11.152 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (3.049 seconds).


Iteration: 0020, time: 3.605 hours, Δt: 7.983 minutes, max(|u|) = 3.6e-01 ms⁻¹, wall time: 14.894 seconds
Iteration: 0040, time: 5.890 hours, Δt: 5.103 minutes, max(|u|) = 5.4e-01 ms⁻¹, wall time: 15.087 seconds
Iteration: 0060, time: 7.386 hours, Δt: 4.325 minutes, max(|u|) = 6.3e-01 ms⁻¹, wall time: 15.280 seconds
Iteration: 0080, time: 8.754 hours, Δt: 3.912 minutes, max(|u|) = 7.0e-01 ms⁻¹, wall time: 15.477 seconds
Iteration: 0100, time: 10 hours, Δt: 3.669 minutes, max(|u|) = 7.7e-01 ms⁻¹, wall time: 15.679 seconds
Iteration: 0120, time: 11.178 hours, Δt: 3.486 minutes, max(|u|) = 8.3e-01 ms⁻¹, wall time: 15.879 seconds
Iteration: 0140, time: 12.284 hours, Δt: 3.330 minutes, max(|u|) = 9.0e-01 ms⁻¹, wall time: 16.069 seconds
Iteration: 0160, time: 13.379 hours, Δt: 3.167 minutes, max(|u|) = 9.9e-01 ms⁻¹, wall time: 16.257 seconds
Iteration: 0180, time: 14.411 hours, Δt: 2.996 minutes, max(|u|) = 1.1e+00 ms⁻¹, wall time: 16.446 seconds
Iteration: 0200, time: 15.387 hours, Δt: 2.81

Excessive output truncated after 524356 bytes.